In [18]:
#import nltk
#nltk.download()

In [19]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import gensim
import gensim.downloader as api
import pandas as pd
from corus import load_wiki
from gensim import corpora
from sklearn.metrics.pairwise import cosine_similarity


In [20]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [21]:
path = r'C:\Users\seismopro\ruwiki-latest-pages-articles.xml.bz2'
records = load_wiki(path)
next(records)

2023-12-15 16:03:01,719 : INFO : 7	Литва


WikiRecord(
    id='7',
    url='?curid=7',
    title='Литва',
    text='Литва\n\nЛитва́ ( ), официальное название\xa0— Лито́вская Респу́блика ()\xa0— государство, расположенное в Северной Европе. Площадь\xa0— км². Протяжённость с севера на юг\xa0— 280\xa0км, а с запада на восток\xa0— 370 км. Население составляет человек (август, 2023). Занимает 137-е место в мире по численности населения и 121-е по территории. Имеет выход к Балтийскому морю, расположена на его восточном побережье. Береговая линия составляет всего 99 км (наименьший показатель среди государств Балтии). На севере граничит с Латвией, на юго-востоке\xa0— с Белоруссией, на юго-западе\xa0— с Польшей и Калининградской областью России. По площади и населению является самым крупным государством из стран Балтии.Столица\xa0— Вильнюс. Официальный язык\xa0— литовский. Денежная единица\xa0— евро.Восстановление независимости страны провозглашено 11 марта 1990\xa0года. 6 сентября 1991\xa0года Государственный совет СССР признал независ

In [22]:
path_rus_corpus = r'Tutorials\russian_corpus.xlsx'

In [23]:
df_path_rus_corpus = pd.read_excel(path_rus_corpus, usecols=[1])

In [24]:
path_rus_corpus_list = df_path_rus_corpus[0].tolist()

In [25]:
rus_corpus = [TaggedDocument(path_rus_corpus_list, 'vocabulary')]

In [38]:
model = Doc2Vec(vector_size=100, min_count=1, epochs=100)
model.build_vocab(rus_corpus)

2023-12-15 16:05:14,942 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d100,n5,w5,s0.001,t3>', 'datetime': '2023-12-15T16:05:14.942471', 'gensim': '4.3.0', 'python': '3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2023-12-15 16:05:14,942 : INFO : collecting all words and their counts
2023-12-15 16:05:14,943 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-12-15 16:05:15,122 : WARNING : More unique tags (9) than documents (1).
2023-12-15 16:05:15,124 : INFO : collected 380900 word types and 9 unique tags from a corpus of 1 examples and 380900 words
2023-12-15 16:05:15,124 : INFO : Creating a fresh vocabulary
2023-12-15 16:05:16,164 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 380900 unique words (100.00% of original 380900, drops 0)', 'datetime': '2023-12-15T16:05:16.164894', 'gensim': '4.3.0', 'py

In [39]:
# Train the Doc2Vec model
model.train(rus_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2023-12-15 16:05:26,744 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 380900 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2023-12-15T16:05:26.744083', 'gensim': '4.3.0', 'python': '3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'train'}
2023-12-15 16:05:26,804 : INFO : EPOCH 0: training on 380900 raw words (10010 effective words) took 0.1s, 172625 effective words/s
2023-12-15 16:05:26,843 : INFO : EPOCH 1: training on 380900 raw words (10010 effective words) took 0.0s, 277165 effective words/s
2023-12-15 16:05:26,874 : INFO : EPOCH 2: training on 380900 raw words (10010 effective words) took 0.0s, 377589 effective words/s
2023-12-15 16:05:26,906 : INFO : EPOCH 3: training on 380900 raw words (10010 effective words) took 0.0s, 387606 effective words/s
2023-12-15 16:05:26,936 : INFO : EPOCH 4:

In [48]:
vector_1 = model.infer_vector(['Второе', 'предложение'])
vector_2 = model.infer_vector(['замечание'])

In [49]:
cosine_similarity([vector_1], [vector_2])

array([[0.9973653]], dtype=float32)

In [42]:
string_1 = 'основной вид учебный деятельность раздел дисциплина формировать вид учебный деятельность объем результат время ч обучение дисциплина раздел основа тестирование рд рд лекция тестирование оценка качество лабораторный занятие по самостоятельный работа раздел формирование рдз лекция реализация стратегия лабораторный занятие тестирование по самостоятельный работа содержание раздел дисциплина раздел основа тестирование тестирование оценка качество по рамка раздел рассматриваться общий теоретический основа процесс тестирование программный обеспечение число механизм выявление формирование оценка качество требование стратегия вид тестирование тема лекция тестирование по ретроспектива основа требование выявление анализ документирование проверка вид стратегия тестирование тема практический занятие отчет дефект гостр исо мэк требование оценка качество система программный обеспечение гостр исолмэк требование оценка качество система программный обеспечение элемент показатель качество управление проект планирование отчетность название лабораторный работа требование выявление оценка качество документирование раздел формирование реализация стратегия тестирование по раздел формировать навык разработка реализация стратегия тестирование рассматриваться специализированный вид тестирование автоматизированный регрессионный тестирование удобство использование тема лекция автоматизация тестирование регрессионный тестирование нефункциональный тестирование тестирование удобство использование название лабораторный работа модульный тестирование регрессионный тестирование тема практический занятие неформальный проверка код оценка качество документированность код'
string_2 = 'основной вид учебный деятельность формировать раздел дисциплина результат вид учебный деятельность ева обучение время ч дисциплина } лекция раздел основа работа растровый d- ¥ рд-1 лабораторный занятие 16 графика дизайн самостоятельный работа лекция раздел основа работа векторный d- ¥ рд-1 лабораторный занятие 16 графика дизайн самостоятельный работа содержание раздел дисциплина раздел основа работа растровый графика дизайн введение изучение растровый графика цветовой модель формат файл тема лекция введение растровый векторный график формат графический файл работа цвет компьютерный график основной определение параметр цвет работа цвет компьютерный графика цветовой модель текст растровый программа название лабораторный работа кадрирование перспектива выравнивание горизонт работа искажение цветокоррекция перевод ч б работа слой маска слой простой коллаж работа корректировать слой цветокоррекция помощь корректировать слой сложный коллаж кисть паттерн работа текст анимация создание макет сайт © чи \ o раздел основа работа векторный графика дизайн введение изучение векторный графика верстка документ подготовка файл печать тема лекция основа разметка верстка документ особенность разработка набор печатный продукция допечатный подготовка файл название лабораторный работа панель инструмент примитив простой операция кривая безье кривой безье инструмент форма узор примитив кривой безье работа текст настройка искривление так далее работа слой направлять сетка выделение контур заливка'
list_1 = string_1.split()
list_2 = string_2.split()
vector_1 = model.infer_vector(list_1)
vector_2 = model.infer_vector(list_2)

In [43]:
print(vector_1, vector_2)

[-6.6490755  -3.6557872  -2.2668748   2.35272    -2.119856    0.53825617
 -1.0207084   2.2169836  -2.185419    0.4217647   1.4868679  -1.9599813
  3.216271   -0.12612006 -0.5298889   0.7039598  -0.99458253  3.415865
  0.87553334  3.8557885  -0.20478833 -0.37841457 -0.46141633  3.6983654
 -0.5118344   0.9471238  -5.2754726   0.7595059  -1.8580594  -7.2112923
  4.5431023   2.0801082  -1.1785842   6.8164268   1.5705634   2.2809353
 -3.2367036  -5.5558157   0.83559966 -2.746815    0.7078302   1.2330245
 -2.3817997  -3.3745904  -3.4477823  -1.1288458   0.36481547  2.5413773
 -2.0812793   2.047      -4.0613613  -2.181909   -1.3845962  -1.0242152
 -0.12095653  0.7175634  -3.5511162   3.1691055  -3.116429    0.82742107
  8.301766    5.5853252  -0.0238711   3.3592603   2.505985    1.7610846
  4.6993666   1.1148492  -0.6928487   0.86568296 -5.872152   -4.141739
 -6.2296505  -1.2045702   5.3263235   4.908863    3.5877688  -5.165172
 -1.4179757   1.852846   -2.275515    3.8754263  -1.4171891   2.3

In [44]:
cosine_similarity([vector_1], [vector_2])

array([[0.99999946]], dtype=float32)

In [45]:
from scipy import spatial

result = 1 - spatial.distance.cosine(vector_1, vector_2)
result

0.999999463558197